# Activity Log Routes

> routes for interacting with the activity log


In [ ]:
# | default_exp routes.activity_log

In [ ]:
# | exporti

import aiohttp

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


In [ ]:
# | export
async def get_activity_log_object_types(auth: dmda.DomoAuth) -> rgd.ResponseGetData:
    """retrieves a list of valid objectTypes that can be used to search the activity_log API"""

    url = "https://domo-dojo.domo.com/api/audit/v1/user-audits/objectTypes"

    return await gd.get_data(url=url, method="GET", auth=auth)

#### sample implementation of get_activity_log_object_types


In [ ]:
import os
import pandas as pd

try:
    token_auth = dmda.DomoTokenAuth(
        domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
        domo_instance="domo-dojo",
    )

    al_objecttype_res = await get_activity_log_object_types(auth=token_auth)

    print(pd.DataFrame(al_objecttype_res.response))

    # use to update `ActivityLog_ObjectType` enum
    # [ f"{objecttype.get('translation').upper().replace(' ', '_')} = '{objecttype.get('type')}'" for objecttype in  al_objecttype_res.response]

except Exception as e:
    print(e)


                  type         translation
0         ACCESS_TOKEN        Access token
1              ACCOUNT             Account
2         ACTIVITY_LOG        Activity log
3                ALERT               Alert
4                  APP                 App
..                 ...                 ...
56                VIEW                View
57        VIRTUAL_USER        Virtual user
58     Workbench_AGENT     Workbench agent
59       Workbench_JOB       Workbench job
60  Workbench_SCHEDULE  Workbench schedule

[61 rows x 2 columns]


In [ ]:
# | export
async def search_activity_log(
    auth: dmda.DomoAuth,
    start_time: int,  # epoch time in milliseconds
    end_time: int,  # epoch time in milliseconds
    maximum: int,
    object_type: str = None,
    session: aiohttp.ClientSession = None,
    debug_api: bool = False,
    debug_loop: bool = False,
) -> rgd.ResponseGetData:
    """loops over activity log api to retrieve audit logs"""

    is_close_session = False

    if not session:
        session = aiohttp.ClientSession()
        is_close_session = True

    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"

    if object_type and object_type != 'ACTIVITY_LOG':
        url = f"{url}/objectTypes/{object_type}"

    fixed_params = {"end": end_time, "start": start_time}

    offset_params = {
        "offset": "offset",
        "limit": "limit",
    }

    def arr_fn(res) -> list[dict]:
        return res.response

    res = await gd.looper(
        auth=auth,
        method="GET",
        url=url,
        arr_fn=arr_fn,
        fixed_params=fixed_params,
        offset_params=offset_params,
        session=session,
        maximum=maximum,
        debug_loop=debug_loop,
        debug_api=debug_api,
    )

    if is_close_session:
        await session.close()

    return res


In [ ]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-dojo",
)

page_res = await search_activity_log(
    object_type= 'PAGE',
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=16,
    debug_loop=False,
)

pd.DataFrame(page_res.response[0:5])


,objectId,objectName
0,-100000,Overview
1,-100001,Alerts
2,-100002,Shared
3,-100003,Favorites
4,-100012,CEO


#### sample implementation of search_activity_log


In [ ]:
import os
import pandas as pd
import datetime as dt

import domolibrary.utils.convert as convert

end_datetime = dt.datetime.today()
start_datetime = end_datetime - dt.timedelta(days=100)

token_auth = dmda.DomoTokenAuth(
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
    domo_instance="domo-dojo",
)

activity_log_res = await search_activity_log(
    object_type= 'ACTIVITY_LOG',
    start_time=convert.convert_datetime_to_epoch_millisecond(start_datetime),
    end_time=convert.convert_datetime_to_epoch_millisecond(end_datetime),
    auth=token_auth,
    maximum=10,
    debug_loop=False,
    debug_api=False,
)

pd.DataFrame(activity_log_res.response[0:5])

,userName,userId,userType,actorName,actorId,actionType,objectName,objectId,objectType,additionalComment,time,eventText,clientId
0,miguel@uaslp.mx,miguel@uaslp.mx,USER,,0,FAILED_LOGIN,,,,miguel@uaslp.mx failed logging in. Logged in f...,1673055975748,Failed login,None
1,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673055929282,Logged in,None
2,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673055450187,Logged in,None
3,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673055431938,Logged in,None
4,Jae Wilson,1893952720,USER,,0,LOGGEDIN,,,,Jae Wilson logged in. Logged in from IP addres...,1673055427438,Logged in,None


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
